# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [89]:
# Install required packages
!pip install requests

# Task
Update the notebook to convert Bitcoin price to any target currency supported by api.coingecko.com instead of only ZAR.

## Identify target currencies

### Subtask:
Determine how the user will specify the target currency (e.g., as an input variable, a list of currencies).


**Reasoning**:

*   List item
*   List item


Define a variable to hold the target currency for the conversion.



In [90]:
target_currency = 'eur'

## Modify api call

### Subtask:
Update the API call to fetch the Bitcoin price in the specified target currency or currencies.


**Reasoning**:
Modify the API call to fetch the Bitcoin price in the specified target currency.



**Reasoning**:
The error indicates that the `requests` library was not imported in the current execution environment. Re-import the necessary libraries.



In [91]:
import requests
import sqlite3
from datetime import datetime

# Read data from CoinGecko API
url = f"https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies={target_currency}"
response = requests.get(url)
data = response.json()

data

{'bitcoin': {'eur': 102390}}

## Update data processing

### Subtask:
Adjust the data processing logic to handle the response from the updated API call, which will contain prices in the target currency or currencies.


**Reasoning**:
Adjust the data processing logic to handle the response from the updated API call, which will contain prices in the target currency.



In [92]:
# Extract price in target currency
target_price = data["bitcoin"][target_currency]
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in {target_currency.upper()}: {target_price}")

Bitcoin price in EUR: 102390


**Reasoning**:
The data processing logic has been adjusted to handle the target currency. The next step is to update the database insertion logic to use the target price and currency, and then verify the database update.



**Reasoning**:
The previous code block failed because the 'prices' table does not exist. This is likely because the notebook was reset or the database file was deleted. I need to recreate the table before attempting to alter or insert into it.



In [93]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    timestamp TEXT,
    usd REAL,
    zar REAL
)
''')
conn.commit()

# Check if the target currency column exists and add it if not
cursor.execute(f"PRAGMA table_info(prices)")
columns = [col[1] for col in cursor.fetchall()]

if target_currency not in columns:
    cursor.execute(f"ALTER TABLE prices ADD COLUMN {target_currency} REAL")
    conn.commit()
    print(f"Added column '{target_currency}' to the prices table.")

# Insert the new row
insert_sql = f"INSERT INTO prices (timestamp, {target_currency}) VALUES (?, ?)"
cursor.execute(insert_sql, (timestamp, target_price))

conn.commit()
conn.close()

# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Get all column names
cursor.execute(f"PRAGMA table_info(prices)")
available_columns = [col[1] for col in cursor.fetchall()]

# Select all columns for the last 5 rows
select_sql = f"SELECT {', '.join(available_columns)} FROM prices ORDER BY timestamp DESC LIMIT 5"
rows = cursor.execute(select_sql).fetchall()

for row in rows:
    timestamp_val = row[0]
    prices_info = []
    for i in range(1, len(available_columns)):
        currency = available_columns[i]
        price = row[i]
        if price is not None:
            prices_info.append(f"Bitcoin Price in {currency.upper()}: {price}")

    if prices_info:
        print(f"{timestamp_val}, {', '.join(prices_info)}")
    else:
        print(f"{timestamp_val}, Bitcoin Price: N/A")

conn.close()

2025-08-13T09:24:33.029022, Bitcoin Price in EUR: 102390.0
2025-08-13T09:24:25.652573, Bitcoin Price in CAD: 165141.0
2025-08-13T09:23:51.632682, Bitcoin Price in USD: 120046.0
2025-08-13T09:23:35.830420, Bitcoin Price in AUD: 183173.0
2025-08-13T09:23:21.427581, Bitcoin Price in ZAR: 2101822.0


## Summary:

### Data Analysis Key Findings

*   The notebook was successfully updated to allow the user to specify a target currency for Bitcoin price conversion using a variable.
*   The API call was modified to dynamically fetch the Bitcoin price in the specified target currency from the CoinGecko API.
*   The data processing logic was adjusted to handle the API response containing the price in the target currency.
*   The SQLite database schema was updated to dynamically add a column for the target currency if it does not exist.
*   The database insertion logic was adjusted to correctly insert the fetched price into the newly added column in the `prices` table.
*   The data display code was modified to dynamically retrieve and display the Bitcoin price in the target currency from the database, along with other available currency columns.
